In [1]:
import sys 
import os 
import datetime

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import matplotlib.ticker as mtick
from IPython.display import display, HTML


if "D:/src/PortAttribute" not in sys.path:
    sys.path.insert(0, "D:/src/PortAttribute")
if "D:/src/Backtest" not in sys.path:
    sys.path.insert(0, "D:/src/Backtest")
if "D:/src/OptPort" not in sys.path:
    sys.path.insert(0, "D:/src/OptPort")
if "D:/src/DataApi" not in sys.path:
    sys.path.insert(0, "D:/src/DataApi")
if "D:/src/" not in sys.path:
    sys.path.insert(0, "D:/src/")
if "D:/Nowcast/" not in sys.path:
    sys.path.insert(0, "D:/Nowcast/")
    
from portattr import plotting, ret_metric, exante
from fast_bt import simple_bt
from optport import max_ret, estimator, risk_parity, risk_budget, max_div, max_div_vol, min_var, max_ret, mean_variance
from dataapi import juyuan
from nowcast import api

In [2]:
plotting.setup_plotting_context()

# 资产价格

In [3]:
DATA_PATH = "D:/data"
def read_data(name):
    data = pd.read_csv(DATA_PATH + "/{}.csv".format(name))
    data['date'] = pd.to_datetime(data['date'].astype(str))
    data = data.rename(columns={'date': '日期'})
    return data

In [4]:
asset_class = {
    'H00906.CSI': '股票',
    'CBA02501.CS': '利率债',
    'CBA02001.CS': '信用债',
    'AU9999.SGE': '黄金',
    'NH0100.NHF': '商品'
}
start_year = '2010'

prc = read_data("AssetClose")
prc = prc.loc[prc['field'].isin(asset_class.keys()), :]\
         .pivot(index='日期', columns='field', values='close')\
         .rename(columns=asset_class)
prc = prc.loc[:, ['股票', '利率债', '信用债', '黄金','商品']]

prc.index.name = 'date'
prc.columns.name = 'sid'
prc = prc.ffill()
daily_ret = prc.pct_change()

In [5]:
bt_p = prc.loc[start_year: ].stack().to_frame('close')
bt_p['open'] = bt_p['close']
bt_p['vwap'] = bt_p['close']
bt_p = bt_p.sort_index()

In [6]:
def month_rebalance_date(dates_list, n=10):
    """
    """
    tmp = pd.DataFrame({'date': dates_list})
    tmp['month'] = tmp['date'].dt.month 
    tmp['year'] = tmp['date'].dt.year
    tmp['rank'] = tmp.groupby(['year', 'month'])['date'].rank().astype(int)
    tmp = tmp.query("rank == {}".format(n))
    return tmp['date'].tolist()

def quarter_rebalance_date(dates_list, month=[2,5,8,11], n=10):
    tmp = pd.DataFrame({'date': dates_list})
    tmp['date'] = pd.to_datetime(tmp['date'])
    tmp['month'] = tmp['date'].dt.month
    tmp['year'] = tmp['date'].dt.year
    tmp = tmp.loc[tmp['month'].isin(month)]
    tmp['rank'] = tmp.groupby(['year', 'month'])['date'].rank().astype(int)
    tmp = tmp.query("rank == {}".format(n))
    return tmp['date'].tolist()

def week_rebalance_date(dates_list, n=1):
    year, med, day = zip(*[x.isocalendar() for x in dates_list])
    tmp = pd.DataFrame({
            'dates': dates_list,
            'year': year,
            'med': med,
            'day': day
        })
    tmp['rank'] = tmp.groupby(["year", "med"])["day"].rank()
    return tmp.query("rank=={}".format(n))['dates'].tolist()

In [7]:
month_dates = month_rebalance_date(prc.loc[start_year:].index.tolist(), n=10)
week_dates = week_rebalance_date(prc.loc[start_year: ].index.tolist())
quarter_dates = quarter_rebalance_date(prc.loc[start_year:].index.tolist(), n=11)

In [8]:
# 组合
port_res = dict()
# 回测
bt_res = dict()
# 净值
nav_res = pd.DataFrame()

In [9]:
def show_bt_ret(start=start_year):
    return pd.DataFrame({
    x: ret_metric.cal_ret_summary(nav_res[x].iloc[1:, ].pct_change().loc[start:], 'daily',rf=0.025) for x in nav_res.columns
})
def show_mean_holding(start=start_year):
    return pd.DataFrame({
        x: port_res[x].loc[start:].mean() for x in port_res
    } )

# 方差估计

In [10]:
var_dict = dict()

for dt in sorted(month_dates + quarter_dates):
    _temp = daily_ret.iloc[max(daily_ret.index.get_loc(dt) - 244 * 3, 0): daily_ret.index.get_loc(dt)]\
                     .loc[:, ['股票', '利率债', '信用债','黄金','商品']]
    cov_es = estimator.exp_cov_estimator(_temp, halflife=126) * 244
    var_dict.update({dt: cov_es})

In [11]:
# _temp_dict = dict()
# for dt in sorted(quarter_dates):
#     cov_es = var_dict[dt]
#     ret_es = pd.Series(np.diag(cov_es), index=cov_es.index) ** 0.5
#     wgt = max_div_vol(var_dict[dt], 0.02, budget=1, max_holding=[0.7, 0.5, 0.3, 0.25])
#     _temp_dict.update({dt: wgt})
# _temp_dict = pd.DataFrame(_temp_dict)
# _temp_dict = _temp_dict.T 
# # _temp_dict = _temp_dict.div(_temp_dict.sum(axis=1), axis=0)
# port_res.update({'最大分散度1.6%': _temp_dict})
# # bt_res.update({
# #     '最大分散度1.6%_m': simple_bt(port_res['最大分散度1.6%'].reindex(month_dates), bt_p, init_cash=1e8)
# # })
# # nav_res['最大分散度3%_m'] = bt_res['最大分散度3%_m'].history_market_value + bt_res['最大分散度3%_m'].history_cash
# bt_res.update({
#     '最大分散度1.6%_q': simple_bt(port_res['最大分散度1.6%'].reindex(quarter_dates), bt_p, init_cash=1e8)
# })
# nav_res['最大分散度1.6%_q'] = bt_res['最大分散度1.6%_q'].history_market_value + bt_res['最大分散度1.6%_q'].history_cash

In [12]:
_temp_dict = dict()
for dt in sorted(quarter_dates + month_dates):
    cov_es = var_dict[dt]
    ret_es = pd.Series(np.diag(cov_es), index=cov_es.index) ** 0.5
    wgt = max_div_vol(var_dict[dt], 0.03, max_budget=1, max_holding=[0.7, 0.5, 0.3, 0.15,0.1], min_holding=[0.05, 0.05, 0.05,0.05, 0.05])
    _temp_dict.update({dt: wgt})
_temp_dict = pd.DataFrame(_temp_dict)
_temp_dict = _temp_dict.T 
# _temp_dict = _temp_dict.div(_temp_dict.sum(axis=1), axis=0)
port_res.update({'最大分散度3%': _temp_dict})
bt_res.update({
    '最大分散度3%_m': simple_bt(port_res['最大分散度3%'].reindex(month_dates), bt_p, init_cash=1e8)
})
nav_res['最大分散度3%_m'] = bt_res['最大分散度3%_m'].history_market_value + bt_res['最大分散度3%_m'].history_cash
bt_res.update({
    '最大分散度3%_q': simple_bt(port_res['最大分散度3%'].reindex(quarter_dates), bt_p, init_cash=1e8)
})
nav_res['最大分散度3%_q'] = bt_res['最大分散度3%_q'].history_market_value + bt_res['最大分散度3%_q'].history_cash

[Fast Backtest] default In Progress: 100%|########################################| 2762/2762 [00:03<00:00, 692.62it/s]


In [13]:
_temp_dict = dict()
for dt in sorted(quarter_dates + month_dates):
    cov_es = var_dict[dt]
    ret_es = pd.Series(np.diag(cov_es), index=cov_es.index) ** 0.5
    wgt = max_div_vol(var_dict[dt], 0.06, max_budget=1, max_holding=[0.7, 0.5, 0.3, 0.15,0.1], min_holding=[0.05, 0.05, 0.05,0.05, 0.05])
    _temp_dict.update({dt: wgt})
_temp_dict = pd.DataFrame(_temp_dict)
_temp_dict = _temp_dict.T 
# _temp_dict = _temp_dict.div(_temp_dict.sum(axis=1), axis=0)
port_res.update({'最大分散度6%': _temp_dict})
bt_res.update({
    '最大分散度6%_m': simple_bt(port_res['最大分散度6%'].reindex(month_dates), bt_p, init_cash=1e8)
})
nav_res['最大分散度6%_m'] = bt_res['最大分散度6%_m'].history_market_value + bt_res['最大分散度6%_m'].history_cash
bt_res.update({
    '最大分散度6%_q': simple_bt(port_res['最大分散度6%'].reindex(quarter_dates), bt_p, init_cash=1e8)
})
nav_res['最大分散度6%_q'] = bt_res['最大分散度6%_q'].history_market_value + bt_res['最大分散度6%_q'].history_cash

[Fast Backtest] default In Progress: 100%|########################################| 2762/2762 [00:04<00:00, 673.72it/s]


In [14]:
_temp_dict = dict()
for dt in sorted(quarter_dates + month_dates):
    cov_es = var_dict[dt]
    ret_es = pd.Series(np.diag(cov_es), index=cov_es.index) ** 0.5
    wgt = max_div_vol(var_dict[dt], 0.1, max_budget=1, max_holding=[0.7, 0.5, 0.3, 0.15,0.1], min_holding=[0.05, 0.05, 0.05,0.05, 0.05])
    _temp_dict.update({dt: wgt})
_temp_dict = pd.DataFrame(_temp_dict)
_temp_dict = _temp_dict.T 
# _temp_dict = _temp_dict.div(_temp_dict.sum(axis=1), axis=0)
port_res.update({'最大分散度10%': _temp_dict})
bt_res.update({
    '最大分散度10%_m': simple_bt(port_res['最大分散度10%'].reindex(month_dates), bt_p, init_cash=1e8)
})
nav_res['最大分散度10%_m'] = bt_res['最大分散度10%_m'].history_market_value + bt_res['最大分散度10%_m'].history_cash
bt_res.update({
    '最大分散度10%_q': simple_bt(port_res['最大分散度10%'].reindex(quarter_dates), bt_p, init_cash=1e8)
})
nav_res['最大分散度10%_q'] = bt_res['最大分散度10%_q'].history_market_value + bt_res['最大分散度10%_q'].history_cash

[Fast Backtest] default In Progress: 100%|########################################| 2762/2762 [00:04<00:00, 682.45it/s]


In [15]:
_temp_dict = dict()
for dt in sorted(quarter_dates + month_dates):
    cov_es = var_dict[dt]
    ret_es = pd.Series(np.diag(cov_es), index=cov_es.index) ** 0.5
    wgt = risk_parity(var_dict[dt])
    _temp_dict.update({dt: wgt})
_temp_dict = pd.DataFrame(_temp_dict)
_temp_dict = _temp_dict.T 
# _temp_dict = _temp_dict.div(_temp_dict.sum(axis=1), axis=0)
port_res.update({'风险平价': _temp_dict})
bt_res.update({
    '风险平价_m': simple_bt(port_res['风险平价'].reindex(month_dates), bt_p, init_cash=1e8)
})
nav_res['风险平价_m'] = bt_res['风险平价_m'].history_market_value + bt_res['风险平价_m'].history_cash
bt_res.update({
    '风险平价_q': simple_bt(port_res['风险平价'].reindex(quarter_dates), bt_p, init_cash=1e8)
})
nav_res['风险平价_q'] = bt_res['风险平价_q'].history_market_value + bt_res['风险平价_q'].history_cash

[Fast Backtest] default In Progress: 100%|########################################| 2762/2762 [00:04<00:00, 687.75it/s]


In [16]:
show_bt_ret()

,最大分散度3%_m,最大分散度3%_q,最大分散度6%_m,最大分散度6%_q,最大分散度10%_m,最大分散度10%_q,风险平价_m,风险平价_q
夏普,0.70,0.72,0.49,0.50,0.40,0.40,1.28,1.24
总收益率,68.1%,69.4%,81.6%,83.2%,96.6%,98.8%,70.3%,69.9%
年化收益率,4.6%,4.7%,5.4%,5.5%,6.4%,6.6%,4.7%,4.7%
年化波动率,3.0%,3.0%,5.9%,6.0%,9.8%,10.1%,1.7%,1.7%
年化下行波动率,2.2%,2.3%,4.3%,4.3%,6.9%,7.1%,1.4%,1.4%
最大回撤,-6.3%,-6.1%,-9.6%,-10.2%,-18.0%,-18.6%,-3.8%,-3.9%
最大回撤期起,2013-02-08,2013-02-08,2013-02-08,2015-06-12,2018-01-24,2015-06-12,2013-05-30,2013-05-30
最大回撤期止,2014-01-09,2014-01-09,2014-01-10,2015-09-15,2018-10-18,2015-09-15,2014-01-09,2014-01-09
最长回撤期起,2013-02-08,2013-02-08,2010-11-09,2010-11-09,2010-11-09,2010-11-09,2010-10-15,2016-11-11
最长回撤期止,2014-10-13,2014-10-08,2013-01-04,2012-12-28,2014-11-26,2014-11-25,2011-11-01,2018-02-23


In [17]:
show_mean_holding()

,最大分散度3%,最大分散度6%,最大分散度10%,风险平价
股票,0.074468,0.208924,0.403498,0.029886
利率债,0.489303,0.395268,0.209510,0.428610
信用债,0.248432,0.146277,0.136992,0.446412
黄金,0.107660,0.149952,0.150000,0.048718
商品,0.074020,0.099579,0.100000,0.046374


In [18]:
ret_metric.cal_yearly_ret_summary(nav_res.pct_change()['最大分散度10%_q'], 'daily')

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
夏普,1.00,-1.06,0.82,-0.55,2.57,0.67,0.76,1.49,-1.15,2.16,1.26,0.66
总收益率,7.4%,-9.7%,7.1%,-5.7%,27.5%,8.0%,7.1%,8.7%,-14.4%,23.2%,14.5%,1.8%
年化收益率,8.3%,-9.2%,7.1%,-5.6%,25.0%,10.4%,5.2%,8.8%,-13.8%,20.9%,15.1%,6.2%
年化波动率,8.3%,8.7%,8.6%,10.1%,9.7%,15.6%,6.8%,5.9%,12.0%,9.6%,11.9%,9.5%
年化下行波动率,5.5%,5.6%,6.7%,6.8%,8.1%,10.3%,4.6%,4.4%,7.7%,7.6%,8.2%,6.8%
最大回撤,-6.0%,-12.0%,-8.0%,-12.6%,-3.9%,-18.6%,-4.3%,-4.2%,-17.8%,-7.0%,-9.7%,-6.1%
最大回撤期起,2010-04-14,2011-04-18,2012-03-13,2013-02-08,2014-02-17,2015-06-12,2016-11-28,2017-04-13,2018-01-24,2019-04-10,2020-03-05,2021-02-19
最大回撤期止,2010-07-05,2011-12-29,2012-12-03,2013-06-27,2014-03-10,2015-09-15,2016-12-23,2017-05-10,2018-10-18,2019-05-09,2020-03-23,2021-03-24
最长回撤期起,2010-04-14,2011-04-18,2012-03-13,2013-02-08,2014-02-17,2015-06-12,2016-08-16,2017-04-13,2018-01-24,2019-04-10,2020-02-24,2021-02-19
最长回撤期止,2010-08-27,2011-12-30,2012-12-31,2013-12-31,2014-07-23,2015-12-31,2016-10-21,2017-06-23,2018-12-28,2019-08-30,2020-06-18,2021-06-25


In [19]:
ret_metric.cal_yearly_ret_summary(nav_res.pct_change()['最大分散度6%_q'], 'daily')

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
夏普,1.52,-0.50,1.16,-1.01,2.89,0.79,1.39,1.36,-0.69,2.63,1.35,0.96
总收益率,7.1%,-3.3%,5.5%,-5.9%,17.7%,5.8%,7.7%,5.3%,-5.5%,16.5%,9.9%,1.9%
年化收益率,7.7%,-2.8%,5.6%,-6.2%,16.7%,6.9%,6.5%,5.3%,-4.7%,15.2%,10.2%,5.7%
年化波动率,5.1%,5.7%,4.8%,6.1%,5.8%,8.6%,4.6%,3.9%,6.8%,5.8%,7.6%,5.9%
年化下行波动率,3.5%,3.7%,3.7%,4.0%,4.9%,5.9%,3.4%,2.9%,4.5%,4.6%,5.3%,4.3%
最大回撤,-3.8%,-5.5%,-3.1%,-9.1%,-1.7%,-10.2%,-3.7%,-3.0%,-8.1%,-4.1%,-6.7%,-3.2%
最大回撤期起,2010-11-09,2011-08-23,2012-02-28,2013-02-08,2014-02-17,2015-06-12,2016-11-28,2017-04-13,2018-01-25,2019-04-10,2020-02-24,2021-02-19
最大回撤期止,2010-11-17,2011-12-29,2012-11-29,2013-12-20,2014-03-10,2015-09-15,2016-12-20,2017-05-10,2018-10-18,2019-05-09,2020-03-23,2021-03-09
最长回撤期起,2010-04-14,2011-08-23,2012-02-28,2013-02-08,2014-04-14,2015-06-12,2016-08-16,2017-04-13,2018-01-25,2019-09-09,2020-08-06,2021-02-19
最长回撤期止,2010-07-30,2011-12-30,2012-12-27,2013-12-31,2014-06-12,2015-12-31,2016-10-21,2017-06-28,2018-12-28,2019-12-16,2020-12-16,2021-05-11


In [20]:
ret_metric.cal_yearly_ret_summary(nav_res.pct_change()['最大分散度3%_q'], 'daily')

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
夏普,1.54,0.54,1.97,-1.38,3.60,1.67,1.92,1.16,1.03,3.64,1.63,1.80
总收益率,3.8%,1.4%,4.4%,-4.2%,10.8%,5.6%,6.0%,2.5%,2.5%,11.3%,6.8%,2.3%
年化收益率,4.1%,1.7%,4.5%,-4.4%,10.5%,5.7%,5.5%,2.5%,2.9%,10.8%,6.9%,5.6%
年化波动率,2.7%,3.2%,2.3%,3.2%,2.9%,3.4%,2.8%,2.2%,2.8%,3.0%,4.2%,3.1%
年化下行波动率,1.8%,2.2%,1.8%,2.0%,2.5%,2.6%,2.2%,1.6%,2.1%,2.5%,3.1%,2.3%
最大回撤,-2.5%,-3.2%,-1.0%,-6.0%,-1.0%,-3.5%,-3.4%,-2.0%,-1.7%,-1.7%,-4.2%,-1.2%
最大回撤期起,2010-11-09,2011-08-23,2012-10-18,2013-02-08,2014-12-04,2015-06-11,2016-11-11,2017-04-13,2018-01-25,2019-04-10,2020-02-24,2021-02-23
最大回撤期止,2010-11-30,2011-09-26,2012-11-29,2013-12-20,2014-12-09,2015-07-08,2016-12-20,2017-05-10,2018-02-09,2019-05-06,2020-03-19,2021-03-09
最长回撤期起,2010-04-26,2011-04-22,2012-07-06,2013-02-08,2014-03-03,2015-06-11,2016-08-16,2017-09-06,2018-04-19,2019-09-05,2020-08-06,2021-02-23
最长回撤期止,2010-07-15,2011-07-12,2012-09-07,2013-12-31,2014-04-11,2015-12-24,2016-10-17,2017-12-29,2018-07-20,2019-12-16,2020-12-15,2021-04-16


In [21]:
ret_metric.cal_yearly_ret_summary(nav_res.pct_change()['风险平价_q'], 'daily')

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
夏普,1.85,1.65,3.30,-1.04,4.88,2.76,2.20,1.14,4.40,6.22,2.88,4.08
总收益率,3.3%,2.8%,4.2%,-1.6%,10.6%,6.7%,4.8%,1.6%,6.1%,7.9%,4.9%,2.4%
年化收益率,3.4%,3.0%,4.2%,-1.7%,10.2%,6.7%,4.4%,1.7%,6.2%,7.8%,4.8%,5.3%
年化波动率,1.8%,1.8%,1.3%,1.6%,2.1%,2.4%,2.0%,1.5%,1.4%,1.3%,1.7%,1.3%
年化下行波动率,1.2%,1.4%,1.1%,1.0%,1.8%,1.9%,1.5%,1.1%,1.3%,1.2%,1.4%,1.1%
最大回撤,-2.6%,-1.7%,-0.7%,-3.8%,-1.4%,-2.3%,-3.4%,-1.7%,-0.8%,-0.5%,-1.3%,-0.5%
最大回撤期起,2010-10-15,2011-08-23,2012-07-16,2013-05-30,2014-12-02,2015-06-11,2016-11-11,2017-01-16,2018-08-07,2019-04-08,2020-03-09,2021-01-21
最大回撤期止,2010-11-26,2011-09-26,2012-09-20,2013-12-23,2014-12-09,2015-07-08,2016-12-20,2017-05-12,2018-08-17,2019-04-29,2020-03-19,2021-01-29
最长回撤期起,2010-10-15,2011-01-12,2012-07-16,2013-05-30,2014-03-03,2015-06-11,2016-11-11,2017-01-16,2018-04-19,2019-10-10,2020-05-06,2021-02-25
最长回撤期止,2010-12-31,2011-04-07,2012-12-07,2013-12-31,2014-04-11,2015-10-13,2016-12-30,2017-06-26,2018-07-06,2019-11-15,2020-07-24,2021-03-25


# BL

In [23]:
dt = pd.Timestamp('20210611')

wb = port_res['最大分散度10%'].loc[dt].round(2)
var = var_dict[dt]

lbd = (0.065 - 0.03)  / 0.1 ** 2 * 2

tau = 0.025

implied_ret = lbd * var.dot(wb)

# p_1 = pd.Series([1, 0, 0, 0], index=var.index)
# p_2 = pd.Series([0, 1, -1, 0], index=var.index)
# q_1 = 0.2
# q_2 = 0.01
# c_1 = 0.5
# c_2 = 0.5

In [94]:
implied_ret

sid
股票     0.131492
利率债   -0.001049
信用债   -0.000037
黄金     0.040846
商品     0.063363
dtype: float64

In [ ]:
def ret100(var, p, q, imp, tau=0.025):
    
    res = imp + tau * var.dot(p) / (p.T.dot(tau * var).dot(p)) * (q - p.T.dot(imp))
    
    return res

In [ ]:
r100_1 = ret100(var, p_1, q_1, implied_ret)
r100_2 = ret100(var, p_2, q_2, implied_ret)

In [ ]:
w100_1 = pd.Series(np.linalg.pinv(var).dot(r100_1) / lbd, wb.index)
w100_2 = pd.Series(np.linalg.pinv(var).dot(r100_2) / lbd, wb.index)

In [ ]:
wk_1 = wb + (w100_1 - wb) * c_1
wk_2 = wb + (w100_2 - wb) * c_2

In [ ]:
(mean_variance(r100_2, lbd/ 2, var, long_only=True, budget=1).round(2)) - wb

In [ ]:
(w100_2 - wb).round(2)

In [25]:
from pypfopt.black_litterman import BlackLittermanModel, market_implied_risk_aversion

In [91]:
confidence = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
imp_r = dict()
all_r = dict()
pre_r = dict()
for i in confidence:
    bl = BlackLittermanModel(cov_matrix=var, pi=implied_ret, P = np.array([[0,0,0,1,-1],
                                                                          [1,0,0,0,0]]), Q=np.array([0.05, 0.2]).reshape(-1, 1), 
                             omega='idzorek', view_confidences=[i, 0.7], risk_aversion=3)
    final_res = pd.Series(bl.bl_weights())
    final_ret = pd.Series(bl.bl_returns())
    r = mean_variance(final_ret, lbd/2, var, long_only=True, budget=1)
    imp_r.update({i: final_ret})
    all_r.update({i: r})
    pre_r.update({i: final_res})
    bl.clean_weights()

In [93]:
pd.DataFrame(pre_r).T.round(3)

,股票,利率债,信用债,黄金,商品
0.1,0.516,0.229,0.042,0.153,0.060
0.2,0.518,0.228,0.042,0.177,0.034
0.3,0.520,0.228,0.042,0.202,0.008
0.4,0.521,0.227,0.042,0.227,-0.017
0.5,0.523,0.226,0.042,0.252,-0.042
0.6,0.525,0.225,0.042,0.276,-0.068
0.7,0.526,0.224,0.042,0.301,-0.093
0.8,0.528,0.224,0.041,0.326,-0.119
0.9,0.530,0.223,0.041,0.350,-0.144


In [84]:
pd.DataFrame(imp_r).T.round(3)

sid,股票,利率债,信用债,黄金,商品
0.1,0.144,-0.001,-0.0,0.046,0.063
0.2,0.143,-0.001,-0.0,0.049,0.059
0.3,0.142,-0.001,0.0,0.052,0.054
0.4,0.140,-0.001,0.0,0.055,0.050
0.5,0.139,-0.001,0.0,0.058,0.045
0.6,0.138,-0.001,0.0,0.061,0.041
0.7,0.137,-0.000,0.0,0.064,0.037
0.8,0.136,-0.000,0.0,0.067,0.032
0.9,0.134,-0.000,0.0,0.070,0.028


In [72]:
test = pd.DataFrame(all_r).T.round(2)
test

,股票,利率债,信用债,黄金,商品
0.1,0.43,0.27,0.05,0.18,0.07
0.2,0.43,0.27,0.05,0.20,0.05
0.3,0.43,0.27,0.05,0.23,0.02
0.4,0.43,0.28,0.04,0.25,0.00
0.5,0.42,0.30,0.00,0.28,0.00
0.6,0.41,0.29,0.00,0.30,0.00
0.7,0.40,0.28,0.00,0.32,0.00
0.8,0.40,0.26,0.00,0.34,0.00
0.9,0.39,0.25,0.00,0.36,0.00


In [ ]:
ret_metric.cal_ret_summary(daily_ret.loc['2010':, '商品'], 'daily', 0.03)

In [ ]:
prc['商品'].dropna().plot(figsize=(12, 8))

In [ ]:
bl = BlackLittermanModel(cov_matrix=var, pi=implied_ret, P = np.array([[0,-1,1,0],
                                                                          [1,0,0,0]]), Q=np.array([0.02, 0.2]).reshape(-1, 1), 
                             omega='idzorek', view_confidences=[0, i], risk_aversion=lbd)
bl.bl_weights()

In [35]:
(0.2546-0.2418) / (0.2928-0.2418)

0.2509803921568628